In [1]:
from dotenv import load_dotenv
import psycopg2
from psycopg2 import sql
import psycopg2.extras
import os
import pandas as pd
import plotly.express as px
load_dotenv("../.env")  # take environment variables from .env.

True

In [2]:
class DatabaseClient:

    def __init__(self):
        self.client = psycopg2.connect(
            dbname=os.getenv("PG_DATABASE"),
            user=os.getenv("PG_USER"),
            host=os.getenv("PG_HOST"),
            password=os.getenv("PG_PASSWORD")
        )

    def query(self, query, params=None):
        cursor = self.client.cursor(cursor_factory=psycopg2.extras.RealDictCursor)
        try:
            cursor.execute(query, params)
            
            if cursor.description:  # Only SELECT queries will have a description
                return cursor.fetchall()
            
            self.client.commit()  # Commit changes for non-SELECT queries

        except Exception as e:
            self.client.rollback()  # Rollback in case of error
            print(f"Error executing query: {e}")
            raise  # or handle exception in another way

        finally:
            cursor.close()

    def query_df(self, query, params=None):
        return pd.DataFrame(self.query(query, params))

client = DatabaseClient()

In [3]:
res = client.query_df("""
    SELECT 
        author_id, 
        forum_user.username,
        COUNT(*) 
    FROM 
        forum_post
    INNER JOIN
        forum_user
    ON
        forum_post.author_id = forum_user.id
    GROUP BY 
        forum_post.author_id, 
        forum_user.username
    ORDER BY COUNT(*) DESC
""")

res  
# for row in res:
#     print(dict(row))
    # print(f'Author ID: {row[0]} - Number of Posts: {row[1]}')

,author_id,username,count
0,1785,thedon,446
1,3740,Mixinthecloud,220
2,3282,kapu,210
3,10702,mikej,191
4,4933,Dangerous,179
...,...,...,...
402,6110,AmazingGato,1
403,9274,Cwidener7,1
404,7008,kbl_guy,1
405,1503,Benbass1991,1


In [29]:
res = client.query_df("""
    SELECT
        replies,
        COUNT(*) AS count
    FROM
        forum_thread
    WHERE
        replies > 0
    GROUP BY
        replies
    ORDER BY
        replies DESC
""")
res.plot(x='replies', y='count', kind='bar', title='Distribution of replies per thread', backend="plotly")

In [ ]:
res = client.query_df("""
    SELECT 
        url,
        title,
        author
    FROM
        forum_thread
    WHERE
        views = 0
""")

res  

In [ ]:
# find the user with the most posts, and the least views
res = client.query_df("""
    WITH user_posts AS (
        SELECT
            forum_user.username,
            COUNT(*) AS num_posts,
            SUM(views) AS total_views,
            (SUM(views) * 1.0 / COUNT(*)) AS avg_views,
            SUM(forum_thread.replies) AS total_replies,
            (SUM(forum_thread.rating) * 1.0 / COUNT(*)) AS avg_rating
        FROM
            forum_thread
        INNER JOIN
            forum_user
        ON
            forum_thread.author_id = forum_user.id
        GROUP BY
            forum_user.username
        ORDER BY
            num_posts DESC,
            avg_views ASC
    )
    SELECT
        *
    FROM
        user_posts
    ORDER BY
        num_posts DESC
    """)
                      
res

In [ ]:
df_fig = res.copy()
df_fig['avg_rating'] = df_fig['avg_rating'].fillna(0) 

px.scatter(
    df_fig, 'avg_views', 'avg_rating', size='num_posts', hover_name='username', log_y=True, log_x=True
)